# How Python works behind the scenes

As you may know python is an interpreted programming language, that means that a lot of things are happening behind the scenes to make our time with python easier and more friendly.

Have you ever asked yourself how much resources are you using to run your python code? Is it using all your powerful cpu cores and all your memory ram?

By default the answer to that is **no**, you are not using all the power your machine has if you are not into multiprocessing. Python only uses one process by default which is using only one core of your cpu, that means that even your machine has 10 cores your python script is only using 1, so actually you are missing a lot of the power your machine is capable of.

Before deep dive into multiprocessing is easier to understand asynchrounous programming.

# ASYNC Concepts

To compare and fully understand how async works in python, we are going to create some functions to be called in both ways, synchronously and asynchronously. 

Before going into practical examples let's clarify some concepts:

* Concurrency:
* Paralellism:

We create a sync function in the traditional way with *def*, async functions are defined with *async def*. For illustrative purposes, we are also going to see the difference between blocking and non-blocking functions.

In [1]:
import multiprocessing as mp
import time
import asyncio
import sys
from termcolor import colored


class AsyncTests:
    """ Sync and Async functions together with block and non-blocking functions to be tested."""
    
    def sync_user_call(self, user, delay):
        time.sleep(delay) 
        print(f"This is user {user} calling the method sync with delay = {delay}")
        return
    
    async def async_user_call(self, user, delay):
        time.sleep(delay) # blocking func
        print(f"This is user {user} calling the method async with a blocking task with delay = {delay}")
        return
        
    async def async_user_call_nonblocking(self, user, delay):
        await asyncio.sleep(delay) # non-blocking func
        print(f"This is user {user} calling the method async with a non-blocking task with delay = {delay}")
        return
    



In [2]:
from IPython.display import Image
Image(url="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*muAffNdekFS8bNH4lTFLmQ.png")

In [3]:
user1 = AsyncTests()
user2 = AsyncTests()
user3 = AsyncTests()

In [5]:
start = time.perf_counter()
user1.sync_user_call(user=1, delay = 2)
user2.sync_user_call(user=2, delay = 2)
user3.sync_user_call(user=3, delay = 2)
end = time.perf_counter()
elapsed_time = end-start
print(colored(f"Total time with sync execution: {round(elapsed_time,2)}", 'blue'))


start = time.perf_counter()
await user1.async_user_call(user=1, delay=2)
await user2.async_user_call(user=2, delay=2)
await user3.async_user_call(user=3, delay=2)
end = time.perf_counter()
elapsed_time = end-start
print(colored(f"Total time with async execution and blocking function: {round(elapsed_time,2)}", 'blue'))

start = time.perf_counter()
await asyncio.gather(user1.async_user_call_nonblocking(user=1, delay=1), 
                     user2.async_user_call_nonblocking(user=2, delay=1),
                     user3.async_user_call_nonblocking(user=3, delay=1))
end = time.perf_counter()
elapsed_time = end-start
print(colored(f"Total time with async execution and non-blocking function: {round(elapsed_time,2)}",'blue'))


This is user 1 calling the method sync with delay = 2
This is user 2 calling the method sync with delay = 2
This is user 3 calling the method sync with delay = 2
Total time with sync execution: 6.01
This is user 1 calling the method async with a blocking task with delay = 2
This is user 2 calling the method async with a blocking task with delay = 2
This is user 3 calling the method async with a blocking task with delay = 2
Total time with async execution and blocking function: 6.02
This is user 1 calling the method async with a non-blocking task with delay = 1
This is user 2 calling the method async with a non-blocking task with delay = 1
This is user 3 calling the method async with a non-blocking task with delay = 1
Total time with async execution and non-blocking function: 1.0


## Key Concepts
To understand the difference between time.sleep() and asyncio.sleep, it is important to understand the following key concepts:

**Blocking vs. Non-Blocking**: A blocking function stops the execution of the entire program until it has completed, while a non-blocking function allows other tasks to be performed during its execution.

**Coroutines**: A coroutine is a special type of function that can be paused and resumed at specific points. Coroutines are used in asyncio to allow the program to switch between tasks as needed.

**Event Loop**: An event loop is a mechanism that allows the program to switch between tasks. In asyncio, the event loop is responsible for scheduling and executing coroutines.

## When to Use Each Function
When deciding whether to use time.sleep() or asyncio.sleep, it is important to consider the following factors:

**Blocking vs. Non-Blocking**: If you need to pause the execution of the entire program, use time.sleep(). If you need to allow other tasks to be performed during the waiting period, use asyncio.sleep.

**Performance**: asyncio.sleep is generally more performant than time.sleep() because it allows the program to switch between tasks as needed.

**Complexity**: asyncio can be more complex to use than standard functions because it requires the use of coroutines and an event loop. If you do not need the benefits of asyncio, it may be simpler to use standard functions.

## Simultaneous instances with async

In [ ]:
import multiprocessing as mp
import time
import asyncio
import sys
from termcolor import colored


class Test:
    def __init__(self) -> None:
        self.value = 0
    
    def counter(self):
        for i in range(10**9):
            self.value += 1
        return
    
    def sync_user_call(self, user, delay, value):
        time.sleep(delay) 
        self.value = value
        print(f"This is user {user} calling the method sync with delay = {delay} and value = {self.value}")
        return
    
    async def async_user_call(self, user, delay, value):
        time.sleep(delay) # blocking func
        self.value = value
        print(f"This is user {user} calling the method async with a blocking task with delay = {delay} and value = {self.value}")
        return
        
    async def async_user_call_nonblocking(self, user, delay, value):
        await asyncio.sleep(delay) # non-blocking func
        self.value = value
        print(f"This is user {user} calling the method async with a non-blocking task with delay = {delay} and value = {self.value}")
        return

In [188]:
user1 = Test()
user2 = Test()
user3 = Test()
print(f"User1 Test instance id: {id(user1)}")
print(f"User2 Test instance id: {id(user2)}")
print(f"User3 Test instance id: {id(user3)} \n")

user3 = user2
print(f"User2 Test instance id: {id(user2)}")
print(f"User3 Test instance id: {id(user3)}")
# print(sys.getrefcount(user2))


User1 Test instance id: 4509475984
User2 Test instance id: 4401084176
User3 Test instance id: 4401085776 

User2 Test instance id: 4401084176
User3 Test instance id: 4401084176


In [189]:
start = time.perf_counter()
user1.sync_user_call(user=1, delay = 1, value=1)
user2.sync_user_call(user=2, delay = 5, value=2)
user3.sync_user_call(user=3, delay = 1, value=3)
end = time.perf_counter()
elapsed_time = end-start
print(colored(f"Total time with sync execution: {elapsed_time}", 'blue'))
print(f" instance id: {id(user3)} user 3 value is: {user3.value}")
print(f" instance id: {id(user2)} user 2 value is: {user2.value}")
print(f" instance id: {id(user1)} user 1 value is: {user1.value}\n")

start = time.perf_counter()
await user1.async_user_call(user=1, delay=1, value = 10)
await user2.async_user_call(user=2, delay=5, value = 20)
await user3.async_user_call(user=3, delay=1, value = 30)
end = time.perf_counter()
elapsed_time = end-start
print(colored(f"Total time with async execution: {elapsed_time}", 'blue'))
print(f" instance id: {id(user3)} user 3 value is: {user3.value}")
print(f" instance id: {id(user2)} user 2 value is: {user2.value}")
print(f" instance id: {id(user1)} user 1 value is: {user1.value}\n")

start = time.perf_counter()
await asyncio.gather(user1.async_user_call_nonblocking(user=1, delay=1, value=100), 
                     user2.async_user_call_nonblocking(user=2, delay=1, value=200),
                     user3.async_user_call_nonblocking(user=3, delay=1, value=300))
end = time.perf_counter()
elapsed_time = end-start
print(colored(f"Total time with async execution: {elapsed_time}",'blue'))
print(f" instance id: {id(user3)} user 3 value is: {user3.value}")
print(f" instance id: {id(user2)} user 2 value is: {user2.value}")
print(f" instance id: {id(user1)} user 1 value is: {user1.value} \n")

start = time.perf_counter()
await asyncio.gather(user1.async_user_call_nonblocking(user=1, delay=3, value=1000), 
                     user2.async_user_call_nonblocking(user=2, delay=10, value=2000),
                     user3.async_user_call_nonblocking(user=3, delay=1, value=3000))
end = time.perf_counter()
elapsed_time = end-start
print(colored(f"Total time with async execution: {elapsed_time}",'blue'))
print(f" instance id: {id(user3)} user 3 value is: {user3.value}")
print(f" instance id: {id(user2)} user 2 value is: {user2.value}")
print(f" instance id: {id(user1)} user 1 value is: {user1.value}\n")

This is user 1 calling the method sync with delay = 1 and value = 1
This is user 2 calling the method sync with delay = 5 and value = 2
This is user 3 calling the method sync with delay = 1 and value = 3
Total time with sync execution: 7.008699291996891
 instance id: 4401084176 user 3 value is: 3
 instance id: 4401084176 user 2 value is: 3
 instance id: 4509475984 user 1 value is: 1

This is user 1 calling the method async with a blocking task with delay = 1 and value = 10
This is user 2 calling the method async with a blocking task with delay = 5 and value = 20
This is user 3 calling the method async with a blocking task with delay = 1 and value = 30
Total time with async execution: 7.01378629100509
 instance id: 4401084176 user 3 value is: 30
 instance id: 4401084176 user 2 value is: 30
 instance id: 4509475984 user 1 value is: 10

This is user 1 calling the method async with a non-blocking task with delay = 1 and value = 100
This is user 2 calling the method async with a non-blockin

In [184]:
import gc

def call():
    instance = Test()
    print(f"instance id: {id(instance)} and value: {instance.value}")
    print(sys.getrefcount(instance)) 
    # print(gc.garbage)
    # gc.collect()
    print(gc.get_count())
    gc.collect()
    print(gc.get_count())
    return instance.value

response1 = call()
response2 = call()
response3 = call()
print(id(response1), id(response2), id(response3))


instance id: 4509472784 and value: 0
2
(375, 0, 0)
(12, 0, 0)
instance id: 4395283792 and value: 0
2
(26, 0, 0)
(0, 0, 0)
instance id: 4401136912 and value: 0
2
(25, 0, 0)
(8, 0, 0)
4362399080 4362399080 4362399080


In [144]:
import gc

gc.collect()

0